In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.ticker import FormatStrFormatter
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times New Roman"]
})
# import seaborn as sns
# sns.set_theme(context="poster", font_scale=1)

sys.path.append("/home/acho/Sync/KiddLab/MSM/src")
from utils.stim_tools import *

import pymc3 as pm
import arviz as az
import theano.tensor as tt

|| MSM INITIALIZATION
Setting time zone...
Setting paths...
Importing sigtools...
Setting up sounddevice...
Importing level adjustments...
Setting eligible BUG words...
Preparing to create long-term spectrum matched noise...
Set values for tone pattern synthesis...
Initialization complete!


# Expt v1.1
### 09/09/21

In [2]:
# Helper function for comparing answers word-by-word
def compare_answers(str1, str2):
    return np.array([int(str1.split(" ")[i] == str2.split(" ")[i])
                     for i in range(len(str1.split(" ")))])

## Load and process data

In [3]:
stim_df = pd.read_csv(PROJ_DIR/"assets"/"stimuli"/"stimulus_database.csv")
data_fnames = sorted(os.listdir(DATA_DIR))

# Read in each file 
all_runs_df = pd.DataFrame()
# for fname in data_fnames:
# for fname in (data_fnames[:-3] + [data_fnames[-1]]):
for fname in data_fnames[:-3]:
# for fname in [data_fnames[-1]]:
    curr_df = pd.read_csv(DATA_DIR/fname)
    merge_df = pd.merge(curr_df, stim_df)
    all_runs_df = pd.concat([all_runs_df, merge_df])
all_runs_df = all_runs_df.reset_index(drop=True)

# Remove practice blocks and convert the block_num data type to int
all_runs_df = all_runs_df[(all_runs_df["block_num"] != "P1") &
                          (all_runs_df["block_num"] != "P2") &
                          (all_runs_df["block_num"] != "P3")].reset_index(drop=True)
all_runs_df.loc[:, "block_num"] = all_runs_df.loc[:, "block_num"].astype(int)

# Modify the speech correct count to proportions
speech_ID_rows = all_runs_df[(all_runs_df["task_type"] == "speech_ID")].index
all_runs_df.loc[speech_ID_rows, "correct"] /= 5

# Get subject IDs and number of subjects
subject_IDs = sorted(list(set(all_runs_df["subject_ID"].values)))
n_subjects = len(subject_IDs)

## Percent correct plots
* Can show individual plots easily

In [ ]:
# Process data
only_target = all_runs_df[all_runs_df["is_target"]]
subject_avg = only_target.groupby(["subject_ID",
                                   "task_type",
                                   "stim_type",
                                   "amplitude"])
group_avg = only_target.groupby(["task_type",
                                 "stim_type",
                                 "amplitude"])
subject_key_list = []
subject_item_list = []
for key, item in subject_avg:
    subject_key_list.append(key)
    subject_item_list.append(item["correct"].values.mean())
subject_key_list = [(row[0], row[1] + "-" + row[2]) for row in subject_key_list]
subject_key_list = [subject_key_list[6*i] for i in range(3*n_subjects)]
subject_item_list = [subject_item_list[6*i:6*(i + 1)] for i in range(3*n_subjects)]
data_by_subject = dict(zip(subject_key_list, subject_item_list))

group_key_list = []
group_item_list = []
for key, item in group_avg:
    group_key_list.append(key)
    group_item_list.append(item["correct"].values.mean())
group_key_list = [(row[0] + "-" + row[1]) for row in group_key_list]
group_key_list = [group_key_list[6*i] for i in range(3)]
group_item_list = [group_item_list[6*i:6*(i + 1)] for i in range(3)]
group_data = dict(zip(group_key_list, group_item_list))

# Set plot elements
x_values = np.arange(5, 31, 5)

subject_symbols = ["s", "^", "v", "d", "*", "h", "X", "P"]
task_colors = ["r", "b", "g"]
subj_markr_dict = dict(zip(subject_IDs, subject_symbols[:n_subjects]))
task_color_dict = dict(zip(group_data.keys(), task_colors))

legend_elements = [Patch(facecolor="r", label="BUG-MD"),
                   Patch(facecolor="b", label="SMN-MD"),
                   Patch(facecolor="g", label="BUG-ID")]

# Make plot
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

# Lines indicating random guessing
# ax.hlines(1/3, 3, 32, linestyle=":", color="m", alpha=1)
# ax.text(32.2, 1/3 - 0.01, "Chance level (MD)")
# ax.hlines(1/8, 3, 32, linestyle=":", color="g", alpha=1)
# ax.text(32.2, 1/8 - 0.01, "Chance level (ID)")

for task in task_color_dict.keys():
    ax.plot(x_values, group_data[task],
            linestyle="-",
            marker="o",
            markersize="6",
            color=task_color_dict[task])

for subject in subject_IDs:
    for task in task_color_dict.keys():
        ax.scatter(x_values, data_by_subject[(subject, task)],
                   s=48,
                   marker=subj_markr_dict[subject],
                   color="w",
                   edgecolor=task_color_dict[task],
                   alpha=0.75)

ax.set_xticks(np.arange(5, 31, 5))
ax.set_xticklabels([r"${:d}^\circ$".format(i) for i in np.arange(5, 31, 5)], fontsize=16)
ax.set_xlim((3, 32))
ax.set_xlabel("Target motion amplitude", fontsize=20)

ax.set_yticks(np.linspace(0.1, 0.9, 9, endpoint=True))
ax.set_yticklabels(["{:.1f}".format(i) for i in np.linspace(0.1, 0.9, 9, endpoint=True)], fontsize=16)
ax.set_ylim((0, 1))
ax.set_ylabel("\% correct", fontsize=20)

ax.grid(linestyle="-", alpha=0.5)
ax.legend(handles=legend_elements, fontsize=14, loc=2)

plt.show()
# plt.savefig("MSM_v1.1_data_individual.pdf", bbox_inches="tight")

In [ ]:
only_BUG_ID = only_target[only_target["task_type"] == "speech_ID"]
gb_BUG_ID = only_BUG_ID.groupby(["subject_ID", "amplitude"])

key_list = []
item_list = []
for key, item in gb_BUG_ID:
    key_list.append(key)
    subj_responses = item["subj_response"].values
    n_trials = len(item["subj_response"])
    patterns = item["pattern"].values
    summed = sum([compare_answers(subj_responses[i], patterns[i])
                  for i in range(len(subj_responses))])/n_trials
    item_list.append(summed)
#     print(key, summed)

amp_list = [5., 10., 15., 20., 25., 30.]

fig, axes = plt.subplots(n_subjects, 6, figsize=(18, 3*n_subjects))

for i, row_ax in enumerate(axes):
    for j, ax in enumerate(row_ax):
        ax.set_xticks(np.arange(1, 6))
        ax.set_yticks(np.linspace(0.1, 0.9, 9, endpoint=True))        
        if i == 0:
            ax.set_title("Target motion = ${:.0f}^\circ$".format(amp_list[j]), fontsize=16)
        
        if i == (n_subjects - 1):
            ax.set_xticklabels(["{:d}".format(ii) for ii in np.arange(1, 6)], fontsize=12)
        else:
            ax.set_xticklabels(())
        
        if j == 0:
            ax.set_yticklabels(["{:.1f}".format(ii) for ii in
                                np.linspace(0.1, 0.9, 9, endpoint=True)], fontsize=12)
            ax.set_ylabel(subject_IDs[i], fontsize=16)
        else:
            ax.set_yticklabels(())
        
        if i != (n_subjects - 1):
            ax.tick_params(axis="x", which="both", top=False, bottom=False, labelbottom=True)
        
        if j != 0:
            ax.tick_params(axis="y", which="both", left=False, right=False, labelleft=True)


for i, key in enumerate(key_list):
    curr_subj = key[0]
    curr_amp  = key[1]
    row = subject_IDs.index(curr_subj)
    col = amp_list.index(curr_amp)
    axes[row, col].bar(np.arange(1, 6), item_list[i], color="g", alpha=0.75)
    axes[row, col].set_xlim(0.25, 5.75)
    axes[row, col].set_ylim(0, 1)
    axes[row, col].grid(linestyle=":")

plt.text(-16.25, -0.25, "Word position", fontsize=20)

plt.show()
# plt.savefig("MSM_v1.1_word_position_by_subject.pdf", bbox_inches="tight")

## Traditional confidence intervals with full pooling

In [ ]:
def compute_CI(task_df, CI_width=0.95):
    from scipy.stats import norm
    q = 0.5 + CI_width/2
    z_alpha = norm.ppf(q)
    n = task_df.groupby("amplitude").count()["run_num"].values[0]
    task_df_group_p = task_df.groupby("amplitude").mean()["correct"].to_frame()
    task_df_arcsin_group_p = 2/np.pi*np.arcsin(np.sqrt(task_df_group_p))
    task_df_arcsin_se = z_alpha*np.sqrt(task_df_arcsin_group_p*(1 - task_df_arcsin_group_p)/n)
    task_df_arcsin_group_p_lb = task_df_arcsin_group_p - task_df_arcsin_se
    task_df_arcsin_group_p_ub = task_df_arcsin_group_p + task_df_arcsin_se
    task_df_arcsin_CI = pd.concat([task_df_arcsin_group_p_lb,
                                   task_df_arcsin_group_p,
                                   task_df_arcsin_group_p_ub], axis=1)
    task_df_arcsin_CI.columns = ["CI_lb", "p", "CI_ub"]
    task_df_backtransform_CI = np.power(np.sin(np.pi/2*task_df_arcsin_CI), 2)
    return task_df_backtransform_CI, task_df_arcsin_group_p

In [ ]:
# Construct a df consisting of only target rows
MD_BUG = \
    all_runs_df[(all_runs_df["is_target"]) &
                (all_runs_df["stim_type"] == "BUG") &
                (all_runs_df["task_type"] == "motion_detection")]
MD_SMN = \
    all_runs_df[(all_runs_df["is_target"]) &
                (all_runs_df["stim_type"] == "SMN") &
                (all_runs_df["task_type"] == "motion_detection")]
SI_BUG = \
    all_runs_df[(all_runs_df["is_target"]) &
                (all_runs_df["stim_type"] == "BUG") &
                (all_runs_df["task_type"] == "speech_ID")]
MD_BUG_backtransform_CI95, MD_BUG_arcsin = compute_CI(MD_BUG)
MD_SMN_backtransform_CI95, MD_SMN_arcsin = compute_CI(MD_SMN)
SI_BUG_backtransform_CI95, SI_BUG_arcsin = compute_CI(SI_BUG)
MD_BUG_arcsin.insert(0, "task_type", 6*["MD_BUG"])
MD_SMN_arcsin.insert(0, "task_type", 6*["MD_SMN"])
SI_BUG_arcsin.insert(0, "task_type", 6*["SI_BUG"])
group_arcsin_p_df = pd.concat([MD_BUG_arcsin, MD_SMN_arcsin, SI_BUG_arcsin]).reset_index()

In [ ]:
colors = ["r", "b", "g"]
CI_container = [MD_BUG_backtransform_CI95,
                MD_SMN_backtransform_CI95,
                SI_BUG_backtransform_CI95]
x = CI_container[0]["p"].index.values

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.set_title("Per Task 95\% Confidence Intervals (n={:d})".format(n_subjects), fontsize=24)
for i, CI in enumerate(CI_container):
    p = CI["p"].values
    p_err = CI.loc[:, ("CI_lb", "CI_ub")].values
    p_err[:, 0] = p - p_err[:, 0]
    p_err[:, 1] = p_err[:, 1] - p
    ax.errorbar(x + 1.25*(i - 1), p, yerr=p_err.T, fmt="-o", color=colors[i])

ax.set_xticks(np.linspace(5, 30, 6))
ax.set_xticklabels([r"${:d}^\circ$".format(int(i)) for i in np.linspace(5, 30, 6)], fontsize=14)
ax.set_xlabel("Target motion amplitude", fontsize=16)
ax.set_ylim((0, 1))
ax.set_yticks(np.linspace(0.1, 0.9, 9))
ax.set_yticklabels(["{:.1f}".format(f) for f in np.linspace(0.1, 0.9, 9)], fontsize=14)
ax.set_ylabel("Proportion correct", fontsize=16)
ax.grid(":")
ax.legend((r"MD-BUG", r"MD-SMN", r"SI-BUG"), loc=2, fontsize=16)

plt.show()
# plt.savefig("MSM_v1.1_CI95.pdf", bbox_inches="tight")

#### RM-ANOVA

In [ ]:
from statsmodels.stats.anova import AnovaRM

rmanova_fit = AnovaRM(data=group_arcsin_p_df,
                      depvar="correct",
                      subject="task_type",
                      within=["amplitude"]).fit()
print(rmanova_fit.summary())

## Get CI for speech ID broken down by word position and motion amplitude
* Still averaged across subjects (group means)
* Here n=60 per point for 5 subjects

In [ ]:
SI_BUG_gb = SI_BUG.groupby(["amplitude"])
n_SI_by_word = SI_BUG_gb.count()["run_num"].values[0]

key_list = []
item_list = []
for key, item in SI_BUG_gb:
    key_list.append(key)
    subj_responses = item["subj_response"].values
    patterns = item["pattern"].values
    summed = sum([compare_answers(subj_responses[i], patterns[i])
                  for i in range(len(subj_responses))])/n_SI_by_word
    item_list.append(summed)

dummy = []
for i in range(len(key_list)):
    dummy.append([key_list[i]] + item_list[i].tolist())
SI_by_word_position = pd.DataFrame(dummy, columns=("amplitude", 1, 2, 3, 4, 5))



# fig, axes = plt.subplots(1, len(SI_by_word_position), figsize=(3*len(SI_by_word_position), 3))

# x = np.array([1., 2., 3., 4., 5.])
# z_alpha = 1.96
# n = 60

# for i in range(len(SI_by_word_position)):
#     curr_SI_p = SI_by_word_position.loc[i][1:6].values
#     curr_SI_p_arcsin = 2/np.pi*np.arcsin(np.sqrt(curr_SI_p))
#     curr_SI_p_arcsin_se = z_alpha*np.sqrt(curr_SI_p_arcsin*(1 - curr_SI_p_arcsin)/n)
#     curr_SI_p_arcsin_lb = curr_SI_p_arcsin - curr_SI_p_arcsin_se
#     curr_SI_p_arcsin_ub = curr_SI_p_arcsin + curr_SI_p_arcsin_se
#     curr_SI_p_lb = np.power(np.sin(np.pi/2*curr_SI_p_arcsin_lb), 2)
#     curr_SI_p_ub = np.power(np.sin(np.pi/2*curr_SI_p_arcsin_ub), 2)
#     curr_SI_p_se_lb = curr_SI_p - curr_SI_p_lb
#     curr_SI_p_se_ub = curr_SI_p_ub - curr_SI_p
#     p_err = np.concatenate([curr_SI_p_se_lb, curr_SI_p_se_ub])
    
# #     axes[i].errorbar(x, curr_SI_p, yerr=p_err, fmt="-o", color="g")
#     axes[i].plot(x, curr_SI_p)
#     axes[i].plot(x, curr_SI_p_lb, "--")
#     axes[i].plot(x, curr_SI_p_ub, ":")
#     axes[i].set_ylim((0, 1))
#     axes[i].grid(":")

# plt.show()

In [ ]:
fig, axes = plt.subplots(1, len(SI_by_word_position), figsize=(3*len(SI_by_word_position), 3))

x = np.array([1., 2., 3., 4., 5.])
for i, row in SI_by_word_position.iterrows():
    y = row[x].values
    axes[i].plot(x, y, "go-")
    
    axes[i].set_title(r"Target motion = ${:d}^\circ$".format(int(row["amplitude"])), fontsize=16)
    axes[i].set_xticks(np.arange(1, 6))
    axes[i].set_xticklabels(["{:d}".format(d) for d in np.arange(1, 6)], fontsize=14)
    axes[i].set_yticks(np.linspace(0.1, 0.7, 8))
    if i != 0:
        axes[i].set_yticklabels(())
    else:
        axes[i].set_yticklabels(["{:.1f}".format(f) for f in np.linspace(0.1, 0.7, 8)], fontsize=14)
        axes[i].set_ylabel("Proportion correct", fontsize=16)
    axes[i].set_ylim((0.05, 0.75))
    axes[i].grid(":")
fig.text(0.475, -0.05, "Word position", fontsize=16)

plt.show()
# plt.savefig("MSM_v1.1_group_word_position.pdf", bbox_inches="tight")

## Look for position effect

In [ ]:
MD_BUG_L = \
    all_runs_df[(all_runs_df["is_target"]) &
                (all_runs_df["stim_type"] == "BUG") &
                (all_runs_df["task_type"] == "motion_detection") &
                (all_runs_df["init_angle"] == -40)].reset_index(drop=True)
MD_BUG_C = \
    all_runs_df[(all_runs_df["is_target"]) &
                (all_runs_df["stim_type"] == "BUG") &
                (all_runs_df["task_type"] == "motion_detection") &
                (all_runs_df["init_angle"] == 0)].reset_index(drop=True)
MD_BUG_R = \
    all_runs_df[(all_runs_df["is_target"]) &
                (all_runs_df["stim_type"] == "BUG") &
                (all_runs_df["task_type"] == "motion_detection") &
                (all_runs_df["init_angle"] == 40)].reset_index(drop=True)
MD_SMN_L = \
    all_runs_df[(all_runs_df["is_target"]) &
                (all_runs_df["stim_type"] == "SMN") &
                (all_runs_df["task_type"] == "motion_detection") &
                (all_runs_df["init_angle"] == -40)].reset_index(drop=True)
MD_SMN_C = \
    all_runs_df[(all_runs_df["is_target"]) &
                (all_runs_df["stim_type"] == "SMN") &
                (all_runs_df["task_type"] == "motion_detection") &
                (all_runs_df["init_angle"] == 0)].reset_index(drop=True)
MD_SMN_R = \
    all_runs_df[(all_runs_df["is_target"]) &
                (all_runs_df["stim_type"] == "SMN") &
                (all_runs_df["task_type"] == "motion_detection") &
                (all_runs_df["init_angle"] == 40)].reset_index(drop=True)
SI_BUG_L = \
    all_runs_df[(all_runs_df["is_target"]) &
                (all_runs_df["stim_type"] == "BUG") &
                (all_runs_df["task_type"] == "speech_ID") &
                (all_runs_df["init_angle"] == -40)].reset_index(drop=True)
SI_BUG_C = \
    all_runs_df[(all_runs_df["is_target"]) &
                (all_runs_df["stim_type"] == "BUG") &
                (all_runs_df["task_type"] == "speech_ID") &
                (all_runs_df["init_angle"] == 0)].reset_index(drop=True)
SI_BUG_R = \
    all_runs_df[(all_runs_df["is_target"]) &
                (all_runs_df["stim_type"] == "BUG") &
                (all_runs_df["task_type"] == "speech_ID") &
                (all_runs_df["init_angle"] == 40)].reset_index(drop=True)

In [ ]:
amplitudes = sorted(list(set(MD_BUG_L["amplitude"].values)))

MD_BUG_L_corr = MD_BUG_L.groupby(["amplitude"]).mean()["correct"].values
MD_BUG_C_corr = MD_BUG_C.groupby(["amplitude"]).mean()["correct"].values
MD_BUG_R_corr = MD_BUG_R.groupby(["amplitude"]).mean()["correct"].values

MD_SMN_L_corr = MD_SMN_L.groupby(["amplitude"]).mean()["correct"].values
MD_SMN_C_corr = MD_SMN_C.groupby(["amplitude"]).mean()["correct"].values
MD_SMN_R_corr = MD_SMN_R.groupby(["amplitude"]).mean()["correct"].values

SI_BUG_L_corr = SI_BUG_L.groupby(["amplitude"]).mean()["correct"].values
SI_BUG_C_corr = SI_BUG_C.groupby(["amplitude"]).mean()["correct"].values
SI_BUG_R_corr = SI_BUG_R.groupby(["amplitude"]).mean()["correct"].values

In [ ]:
fig, axes = plt.subplots(3, len(amplitudes), figsize=(3*len(amplitudes), 10))

for i, amp in enumerate(amplitudes):
    axes[0, i].bar(-1, MD_BUG_L_corr[i], color="b")
    axes[0, i].bar( 0, MD_BUG_C_corr[i], color="g")
    axes[0, i].bar( 1, MD_BUG_R_corr[i], color="r")
    axes[0, i].set_xticks((-1, 0, 1))
    axes[0, i].set_xticklabels(["$-40^\circ$", "$0^\circ$", "$40^\circ$"])
    axes[0, i].set_yticks(np.linspace(0.1, 0.9, 9))
    axes[0, i].set_yticklabels(["{:.1f}".format(num) for num in np.linspace(0.1, 0.9, 9)])
    axes[0, i].set_ylim((0, 1))
    axes[0, i].grid(":", alpha=0.5)

    axes[1, i].bar(-1, MD_SMN_L_corr[i], color="b")
    axes[1, i].bar( 0, MD_SMN_C_corr[i], color="g")
    axes[1, i].bar( 1, MD_SMN_R_corr[i], color="r")
    axes[1, i].set_xticks((-1, 0, 1))
    axes[1, i].set_xticklabels(["$-40^\circ$", "$0^\circ$", "$40^\circ$"])
    axes[1, i].set_yticks(np.linspace(0.1, 0.9, 9))
    axes[1, i].set_yticklabels(["{:.1f}".format(num) for num in np.linspace(0.1, 0.9, 9)])
    axes[1, i].set_ylim((0, 1))
    axes[1, i].grid(":", alpha=0.5)
    
    axes[2, i].bar(-1, SI_BUG_L_corr[i], color="b")
    axes[2, i].bar( 0, SI_BUG_C_corr[i], color="g")
    axes[2, i].bar( 1, SI_BUG_R_corr[i], color="r")
    axes[2, i].set_xticks((-1, 0, 1))
    axes[2, i].set_xticklabels(["$-40^\circ$", "$0^\circ$", "$40^\circ$"])
    axes[2, i].set_yticks(np.linspace(0.1, 0.9, 9))
    axes[2, i].set_yticklabels(["{:.1f}".format(num) for num in np.linspace(0.1, 0.9, 9)])
    axes[2, i].set_ylim((0, 1))
    axes[2, i].grid(":", alpha=0.5)

for i in range(3):
    for j in range(6):
        if i == 0:
            axes[i, j].set_title("Target motion = {:d}$^\circ$".format(int(amplitudes[j])))
        if i != 2:
            axes[i, j].set_xticklabels(())
        if j == 0:
            axes[0, j].set_ylabel("MD-BUG", fontsize=16)
            axes[1, j].set_ylabel("MD-SMN", fontsize=16)
            axes[2, j].set_ylabel("SI-BUG", fontsize=16)
        if j != 0:
            axes[i, j].set_yticklabels(())

fig.text(0.45, 0.075, "Target position", fontsize="16")

plt.show()
# plt.savefig("MSM_v1.1_position_effect.pdf", bbox_inches="tight")

## Error analysis

In [ ]:
speech_ID = all_runs_df[(all_runs_df["stim_type"] == "BUG") &
                        (all_runs_df["task_type"] == "speech_ID")].reset_index(drop=True)

total_possible_score = int(len(speech_ID)/3*5/6)
target_stream_by_amp = {5.:0, 10.:0, 15.:0, 20.:0, 25.:0, 30.:0}
masker_stream_by_amp = {5.:0, 10.:0, 15.:0, 20.:0, 25.:0, 30.:0}
for i in range(0, len(speech_ID), 3):
    curr_trial = speech_ID.loc[i:i + 2]
    target_idx  = np.argwhere(curr_trial["is_target"].values)[0][0]
    masker_idxs = sorted(list(set(range(3)) - set([target_idx])))
    curr_amp = curr_trial.iloc[target_idx]["amplitude"]
    subj_response = curr_trial["subj_response"].values[0]
    talker_sentences = [row[1]["pattern"] for
                        row in curr_trial.iterrows()]
    target_stream_by_amp[curr_amp] += sum(compare_answers(subj_response, talker_sentences[target_idx]))
    masker_stream_by_amp[curr_amp] += sum(compare_answers(subj_response, talker_sentences[masker_idxs[0]]) + \
                                          compare_answers(subj_response, talker_sentences[masker_idxs[1]]))

In [ ]:
target_stream_prop = np.array(list(target_stream_by_amp.values()))/total_possible_score
masker_stream_prop = np.array(list(masker_stream_by_amp.values()))/total_possible_score
neither_stream_prop = np.ones(target_stream_prop.size) - target_stream_prop - masker_stream_prop

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.set_title("Error source plot", fontsize=24)

ax.bar(amplitudes, target_stream_prop, width=3, label="Target stream")
ax.bar(amplitudes, masker_stream_prop, width=3, bottom=target_stream_prop, label="Masker stream")
ax.bar(amplitudes, neither_stream_prop, width=3, bottom=target_stream_prop + masker_stream_prop, label="Random error")

ax.set_xlabel("Target motion range", fontsize=18)
ax.set_xticks(amplitudes)
ax.set_xticklabels(["{:d}$^\circ$".format(int(amp)) for amp in amplitudes], fontsize=16)

ax.set_yticks(np.linspace(0.1, 1, 10))
ax.set_yticklabels(["{:.1f}".format(num) for num in np.linspace(0.1, 1, 10)], fontsize=16)

ax.legend(fontsize=16, loc=4)
ax.grid(":", alpha=0.75)

plt.show()
# plt.savefig("MSM_v1.1_error_source.pdf", bbox_inches="tight")

## More detailed error analysis

In [4]:
def get_stream_idx(subj_response, streams):
    "Return the stream index for each word in subject response"
    container = []
    for i, word in enumerate(subj_response.split()):
        curr_pos_words = [streams[j].split()[i] for j in range(len(streams))]
        try:
            curr_word_idx = curr_pos_words.index(word)
        except ValueError:
            curr_word_idx = 1j
        container.append(curr_word_idx)
    return np.array(container)

In [5]:
speech_ID = all_runs_df[(all_runs_df["stim_type"] == "BUG") &
                        (all_runs_df["task_type"] == "speech_ID")].reset_index(drop=True)

In [6]:
np.arange(0, 5400, 3)

array([   0,    3,    6, ..., 5391, 5394, 5397])

In [7]:
len(speech_ID)

5400

In [8]:
zdx = 12
curr_trial = speech_ID.loc[zdx:zdx + 2]
target_idx = curr_trial["is_target"]
subj_response = curr_trial["subj_response"].values[0]
streams = list(curr_trial["pattern"].values)
stream_idx = get_stream_idx(subj_response, streams)

In [9]:
stream_idx

array([0.+1.j, 0.+1.j, 2.+0.j, 0.+0.j, 1.+0.j])

In [ ]:
np.sum(stream_idx)

In [ ]:
np.diff(stream_idx)

In [ ]:
speech_ID = all_runs_df[(all_runs_df["stim_type"] == "BUG") &
                        (all_runs_df["task_type"] == "speech_ID")].reset_index()

total_possible_score = int(len(speech_ID)/3*5/6)
target_stream_by_amp = {5.:0, 10.:0, 15.:0, 20.:0, 25.:0, 30.:0}
masker_stream_by_amp = {5.:0, 10.:0, 15.:0, 20.:0, 25.:0, 30.:0}
for i in range(0, len(speech_ID), 3):
    curr_trial = speech_ID.loc[i:i + 2]
    target_idx  = np.argwhere(curr_trial["is_target"].values)[0][0]
    masker_idxs = sorted(list(set(range(3)) - set([target_idx])))
    curr_amp = curr_trial.iloc[target_idx]["amplitude"]
    subj_response = curr_trial["subj_response"].values[0]
    talker_sentences = [row[1]["pattern"] for
                        row in curr_trial.iterrows()]
    target_stream_by_amp[curr_amp] += sum(compare_answers(subj_response, talker_sentences[target_idx]))
    masker_stream_by_amp[curr_amp] += sum(compare_answers(subj_response, talker_sentences[masker_idxs[0]]) + \
                                          compare_answers(subj_response, talker_sentences[masker_idxs[1]]))

## Logistic regression

In [ ]:
BUG_MD_only_target = \
    all_runs_df[(all_runs_df["stim_type"] == "BUG") &
                (all_runs_df["task_type"] == "motion_detection") &
                (all_runs_df["is_target"]) & 
                (all_runs_df["block_num"] != "P1") &
                (all_runs_df["block_num"] != "P2") &
                (all_runs_df["block_num"] != "P3")]
amps = BUG_MD_only_target["amplitude"].values
X = amps.reshape(len(amps), 1)
# X = np.stack((np.ones(amps.shape), amps)).T
Y = BUG_MD_only_target["correct"].values.astype(int)

n_features = X.shape[1]
with pm.Model() as BUG_MD_model:
    beta = pm.Normal("beta", 0, 10, shape=n_features)
    pred = tt.sum(X*beta, axis=1)
    likelihood = pm.Bernoulli("y", pm.math.invlogit(pred), observed=Y)
    
    trace = pm.sample(5000, chains=2, target_accept=0.9, return_inferencedata=False)

In [ ]:
with BUG_MD_model:
    print(pm.summary(trace))

In [ ]:
with BUG_MD_model:
    az.plot_trace(trace)
    plt.show()

In [ ]:
with BUG_MD_model:
    az.plot_autocorr(trace)
    plt.show()

In [ ]:
with BUG_MD_model:
    az.plot_forest(trace)
    plt.show()

# Pilot v8.2
### 06/17/21

* Score according to amplitude and task_type
* Score according to position and task_type
* Score according to word position and task_type
* Different symbols for different listeners

In [ ]:
only_target_df = all_runs_df[all_runs_df["is_target"]]

# task_type_order = ["motion_detection", "speech_intelligibility"]
# task_type_order = ["motion_detection"]
stim_type_order = ["SMN", "BUG"]
listener_order = ["DC", "AYC"]
colors = ["b", "r"]
symbols = ["s", "^"]
lines = [":", "-."]
# listener_order = ["L452", "L469"]

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

for i, stim_type in enumerate(stim_type_order):
# for task_type in task_type_order:
    curr_task_df = only_target_df[only_target_df["stim_type"] == stim_type]
    curr_amplitudes = sorted(list(set(curr_task_df["amplitude"].values)))
    for j, listener in enumerate(listener_order):
        curr_listener_df = curr_task_df[curr_task_df["subject_ID"] == listener]
        if task_type == "motion_detection":
            grouped_by_amp = curr_listener_df.groupby("amplitude").mean()
            corr = grouped_by_amp["correct"].values
            
            ax.plot(curr_amplitudes, corr, colors[i] + symbols[j] + lines[i], alpha=0.75, markersize=12)

ax.set_xlabel("Target oscillation amplitude [deg]", fontsize=18)
ax.set_ylim((0, 1))
ax.set_yticks(np.arange(0, 1.01, 0.1))
ax.set_ylabel("Proportion correct", fontsize=18)
ax.grid(linestyle=":")
ax.legend(("SMN-DC", "SMN-AYC", "BUG-DC", "BUG-AYC"), fontsize=16)

plt.show()

In [ ]:
grouped_by_amp

# Pilot v8.1
### 06/17/21

In [ ]:
speech_ID = all_runs_df[(all_runs_df["is_target"]) &
                        (all_runs_df["task_type"] == "speech_intelligibility")].drop(\
                columns=["n_src", "src", "task_type", "elapsed_time", "rate", "run_num", "block_num",
                         "trial_num", "stim_num", "is_target"])
speech_ID = speech_ID.reset_index(drop=True)


subjs = ["L469", "L452"]
amps = [0.0, 1.25, 5.0, 15.0, 20.0, 30.0]
data_dict = {}
for subj in subjs:
    for amp in amps:
        curr_corr = speech_ID[(speech_ID["subject_ID"] == subj) &
                              (speech_ID["amplitude"] == amp)]["correct"].values
        curr_n_trials = len(curr_corr)
        curr_corr_prop = sum(curr_corr)/(5*curr_n_trials)
        
        data_dict[(subj, amp)] = (curr_n_trials, curr_corr_prop)

data_dict

In [ ]:
colors = ["r", "b"]

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

for i, subj in enumerate(subjs):
    curr_subj_prop = []
    curr_n_trials = []
    for amp in amps:
        curr_n_trials.append(data_dict[(subj, amp)][0])
        curr_subj_prop.append(data_dict[(subj, amp)][1])
    curr_marker_sizes = np.array(curr_n_trials)
    curr_marker_sizes = 80*curr_marker_sizes/curr_marker_sizes.max()
    ax.plot(amps, curr_subj_prop, colors[i] + "--", alpha=0.75)
    ax.scatter(amps, curr_subj_prop, s=curr_marker_sizes, c=colors[i])

ax.set_title("Speech ID performance", fontsize=24)
ax.set_xlabel("Target motion range [deg]", fontsize=20)
ax.set_ylabel("Proportion correct", fontsize=20)

ax.set_xticks(np.arange(0, 31, 5))
ax.set_xticklabels(["{:d}".format(i) for i in np.arange(0, 31, 5)], fontsize=16)
ax.set_yticks(np.linspace(0, 1, 11))
ax.set_yticklabels(["{:.1f}".format(i) for i in np.linspace(0, 1, 11)], fontsize=16)

ax.grid(linestyle=":")

plt.show()

# Pilot v7.2

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

color_dict = {2: "b", 3: "y", 4: "r"}
x_pos_dict = {0.0: 0, 1.25: 1, 5.: 2, 15.: 3, 20.: 4, 30.:5}
x_ticks = np.array(list(x_pos_dict.values())) - 1.5
x_ticklabels = [r"$0^{\circ}$",
                r"$2.5^{\circ}$",
                r"$10^{\circ}$",
                r"$30^{\circ}$",
                r"$40^{\circ}$",
                r"$60^{\circ}$"]
y_ticks = np.linspace(0.1, 0.9, 9)
legend0 = []
legend0.append(Patch(facecolor="b", edgecolor="b", label="2 talkers", alpha=0.7))
legend0.append(Patch(facecolor="y", edgecolor="y", label="3 talkers", alpha=0.7))
legend0.append(Patch(facecolor="r", edgecolor="r", label="4 talkers", alpha=0.7))

for row in prop_corr_df.iterrows():
    curr_n_src = int(row[1][0])
    curr_amp  = row[1][1]
    curr_prop = row[1][2]
    x_pos = (x_pos_dict[curr_amp] - 1.5) + 0.25*(curr_n_src - 3)
    ax.bar(x_pos, curr_prop, align="center", width=0.2, alpha=0.7, color=color_dict[curr_n_src])
for curr_n_src in color_dict.keys():
    curr_src_df = prop_corr_df[prop_corr_df["n_src"] == curr_n_src]
    x = np.arange(len(x_pos_dict)) - 1.5 + 0.25*(curr_n_src - 3)
    y = curr_src_df["prop_corr"].values
    ax.plot(x, y, alpha=0.9, linestyle="--", linewidth=1.5, color=color_dict[curr_n_src],
                 marker="o", markersize=14,
                 markeredgecolor="k", markeredgewidth=2,
                 markerfacecolor=color_dict[curr_n_src])
ax.set_title("Speech intelligibility as a function of N talkers and movement range", fontsize=20)
ax.set_xticks(x_ticks)
ax.set_xticklabels(x_ticklabels, fontsize=16)
ax.set_xlabel("Target movement range", fontsize=18)
ax.set_xlim((-2, 4))
ax.set_yticks(y_ticks)
ax.set_yticklabels(("{:.1f}".format(i) for i in y_ticks), fontsize=16)
ax.set_ylabel("Proportion correct", fontsize=18)
ax.set_ylim((0, 1))
ax.grid(linestyle=":", alpha=1)
ax.legend(handles=legend0, loc=2, fontsize=16)

plt.show()
# plt.savefig("a.pdf")

# Pilot v7.1

### Plot as a function of rate and n_src

In [ ]:
prop_corr_series = cleaned_df.groupby(["n_src", "amplitude"]).mean()["correct"]/5
prop_corr_df = prop_corr_series.to_frame()
prop_corr_df = prop_corr_df.stack().reset_index()
prop_corr_df = prop_corr_df.drop(columns="level_2")
prop_corr_df.columns = ["n_src", "amplitude", "prop_corr"]

adj_corr = []
for row in prop_corr_df.iterrows():
    curr_src = row[1][0]
    curr_corr = row[1][2]
    adj_corr.append(curr_corr - 1/curr_src)
prop_corr_df.insert(len(prop_corr_df.columns), "adj_corr", adj_corr)
adj_corr_df = prop_corr_df[prop_corr_df["amplitude"] != 0.0]
adj_corr_df = adj_corr_df.drop(columns="prop_corr")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
fig.suptitle("Speech intelligibility as a function of N talkers and movement range", fontsize=30)

### Subplot 0
color_dict = {2: "b", 3: "y", 4: "r"}
x_pos_dict = {0.0: 0, 1.25: 1, 5.: 2, 15.: 3, 20.: 4, 30.:5}
# x_pos_dict = {0.0: 0, 1.25: 1, 5.: 2, 20.: 3}
# x_pos_dict = {0.0: 0, 15.: 1, 30.: 2}
x_ticks = np.array(list(x_pos_dict.values())) - 1.5
x_ticklabels = [r"$0^{\circ}$",
                r"$2.5^{\circ}$",
                r"$10^{\circ}$",
                r"$30^{\circ}$",
                r"$40^{\circ}$",
                r"$60^{\circ}$"]
# x_ticklabels = [r"$0^{\circ}$",
#                 r"$2.5^{\circ}$",
#                 r"$10^{\circ}$",
#                 r"$40^{\circ}$"]
# x_ticklabels = [r"$0^{\circ}$",
#                 r"$15^{\circ}$",
#                 r"$30^{\circ}$"]
y_ticks = np.linspace(0.1, 0.9, 9)
legend0 = []
legend0.append(Patch(facecolor="b", edgecolor="b", label="2 talkers", alpha=0.7))
legend0.append(Patch(facecolor="y", edgecolor="y", label="3 talkers", alpha=0.7))
legend0.append(Patch(facecolor="r", edgecolor="r", label="4 talkers", alpha=0.7))
# legend0.append(Line2D([], [], linestyle="", marker="d", markeredgewidth=1.5,
#                       markeredgecolor="k", markerfacecolor="w", markersize=12,
#                       label="Noise masker"))
# legend0.append(Line2D([], [], linestyle="", marker="s", markeredgewidth=1.5,
#                       markeredgecolor="k", markerfacecolor="w", markersize=12,
#                       label="Speech masker"))
# legend0.append(Line2D([], [], linestyle="", marker="^", markeredgewidth=1.5,
#                       markeredgecolor="k", markerfacecolor="k", markersize=12,
#                       label=r"$\pm 90^{\circ}$"))

for row in prop_corr_df.iterrows():
    curr_n_src = int(row[1][0])
    curr_amp  = row[1][1]
    curr_prop = row[1][2]
    x_pos = (x_pos_dict[curr_amp] - 1.5) + 0.25*(curr_n_src - 3)
    axes[0].bar(x_pos, curr_prop, align="center", width=0.2, alpha=0.7, color=color_dict[curr_n_src])
for curr_n_src in color_dict.keys():
    curr_src_df = prop_corr_df[prop_corr_df["n_src"] == curr_n_src]
    x = np.arange(len(x_pos_dict)) - 1.5 + 0.25*(curr_n_src - 3)
    y = curr_src_df["prop_corr"].values
    axes[0].plot(x, y, alpha=0.9, linestyle="--", linewidth=1.5, color=color_dict[curr_n_src],
                 marker="o", markersize=14,
                 markeredgecolor="k", markeredgewidth=2,
                 markerfacecolor=color_dict[curr_n_src])
# axes[0].hlines([1/2, 1/3, 1/4], -2, 2, linestyle=":",
#                colors=["b", "y", "r"])
axes[0].set_title("Raw proportions correct", fontsize=20)
axes[0].set_xticks(x_ticks)
axes[0].set_xticklabels(x_ticklabels, fontsize=16)
axes[0].set_xlim((-2, 4))
axes[0].set_yticks(y_ticks)
axes[0].set_yticklabels(("{:.1f}".format(i) for i in y_ticks), fontsize=16)
axes[0].set_ylabel("Proportion correct", fontsize=18)
axes[0].set_ylim((0, 1))
axes[0].grid(linestyle=":", alpha=1)
axes[0].legend(handles=legend0, loc=2, fontsize=16)

### Subplot 1
x_pos_dict = {1.25: 0, 5.: 1, 20.: 2}
# x_ticks = np.array(list(x_pos_dict.values())) - 1
# x_ticklabels = [r"$2.5^{\circ}$",
#                 r"$10^{\circ}$",
#                 r"$40^{\circ}$"]
# y_ticks = np.linspace(0.1, 0.8, 8)

# for row in adj_corr_df.iterrows():
#     curr_n_src = int(row[1][0])
#     curr_amp  = row[1][1]
#     curr_prop = row[1][2]
#     x_pos = (x_pos_dict[curr_amp] - 1) + 0.25*(curr_n_src - 3)
#     axes[1].bar(x_pos, curr_prop, align="center", width=0.2, alpha=0.75, color=color_dict[curr_n_src])
# for curr_n_src in color_dict.keys():
#     curr_src_df = adj_corr_df[adj_corr_df["n_src"] == curr_n_src]
#     x = np.arange(len(x_pos_dict)) - 1 + 0.25*(curr_n_src - 3)
#     y = curr_src_df["adj_corr"].values
#     axes[1].plot(x, y, alpha=0.75, linestyle="--", linewidth=1.5, color=color_dict[curr_n_src],
#                  marker="o", markersize=16,
#                  markeredgecolor="k", markeredgewidth=2,
#                  markerfacecolor=color_dict[curr_n_src])
# axes[1].set_title("Random-guess adjusted proportions correct", fontsize=20)
# axes[1].set_xticks(x_ticks)
# axes[1].set_xticklabels(x_ticklabels, fontsize=16)
# axes[1].set_yticks(y_ticks)
# axes[1].set_yticklabels(("{:.1f}".format(i) for i in y_ticks), fontsize=16)
# axes[1].set_ylabel(r"$\Delta$ proportion correct", fontsize=18)
# axes[1].set_ylim((0, 0.75))
# axes[1].grid(linestyle=":", alpha=1)
# axes[1].legend(handles=legend0, loc=2, fontsize=16)

fig.text(0.5, 0.05,
         "Target movement range", ha="center", fontsize=20)
plt.show()
# plt.savefig("a.pdf")

### Plot as a function of starting location

In [ ]:
nonstationary_df = cleaned_df[cleaned_df["amplitude"] != 0]
amp_series = nonstationary_df.groupby(["n_src", "init_angle"]).mean()["correct"]/5
amp_df = amp_series.to_frame()
amp_df = amp_df.stack().reset_index()
amp_df = amp_df.drop(columns="level_2")
amp_df.columns = ["n_src", "amplitude", "prop_corr"]

In [ ]:
color_dict = {2: "b", 3: "y", 4: "r"}
x_pos_dict = {2: -1, 3: 0, 4: 1.25}
x_ticks = (-4.5, -3.5, -1, 0, 1, 3.5, 4.5, 5.5, 6.5)
x_ticklabels = (r"$-20^{\circ}$", r"$+20^{\circ}$",
                r"$-40^{\circ}$", r"$0^{\circ}$", r"$+40^{\circ}$",
                r"$-60^{\circ}$", r"$-20^{\circ}$", r"$+20^{\circ}$", r"$+60^{\circ}$")
y_ticks = np.linspace(0.1, 0.9, 9)
y_ticklabels = ["{:.1f}".format(val) for val in y_ticks]
legend0 = []
legend0.append(Patch(facecolor="b", edgecolor="b", label="2 talkers", alpha=0.7))
legend0.append(Patch(facecolor="y", edgecolor="y", label="3 talkers", alpha=0.7))
legend0.append(Patch(facecolor="r", edgecolor="r", label="4 talkers", alpha=0.7))

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.set_title("Performance as a function of target angle", fontsize=26)
for row in amp_df.iterrows():
    curr_n_src = int(row[1][0])
    curr_init_angle = row[1][1]
    curr_prop = row[1][2]
    x_pos = 4*x_pos_dict[curr_n_src] + curr_init_angle/40
    ax.bar(x_pos, curr_prop, align="center", width=0.8, alpha=0.7, color=color_dict[curr_n_src])
ax.set_xticks(x_ticks)
ax.set_xticklabels(x_ticklabels, fontsize=16)
ax.set_xlabel("Mean target angle", fontsize=18)
ax.set_xlim((-6, 8))
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_ticklabels, fontsize=16)
ax.set_ylabel("Proportion correct", fontsize=18)
ax.set_ylim((0., 1))
ax.grid(linestyle=":")
ax.legend(handles=legend0, loc=1, fontsize=16)

plt.show()
# plt.savefig("b.pdf")

### Plot as a function of word position

In [ ]:
answers = cleaned_df["pattern"].values
responses = cleaned_df["subj_response"].values

corr_list = []
for i in range(len(answers)):
    curr_ans = answers[i].split(" ")
    curr_res = responses[i].split(" ")
    corr_arr = np.array([curr_ans[j] == curr_res[j] for j in range(len(curr_ans))], dtype=int)
    corr_list.append(corr_arr)

class Container:
    def __init__(self, arr):
        self.arr = arr
    
    def __add__(self, other):
        return Container(self.arr + other.arr)
    
    def __mult__(self, other):
        return Container(self.arr*other)
    
    def __div__(self, other):
        return Container(self.arr/other)


corr_list = [Container(arr) for arr in corr_list]

In [ ]:
word_order_df = cleaned_df.drop(columns=["subj_response", "run_num", "block_num", "trial_num",
                                         "correct", "pattern", "init_angle"])
word_order_df.insert(len(word_order_df.columns), "word_order", corr_list)
word_order_series = word_order_df.groupby(["n_src", "amplitude"]).sum()["word_order"]
word_order_df = word_order_series.to_frame()
word_order_df = word_order_df.stack().reset_index()
word_order_df = word_order_df.drop(columns="level_2")
word_order_df.columns = ["n_src", "amplitude", "word_order"]

In [ ]:
color_dict = {2: "b", 3: "y", 4: "r"}
subplot_row_dict = {2: 2, 3: 1, 4: 0}
subplot_col_dict = {0.0: 0, 1.25: 1, 5.: 2, 20.: 3}
x_pos = np.array([1, 2, 3, 4, 5])

fig, axes = plt.subplots(3, 4, figsize=(12, 9))
fig.suptitle("Performance as a function of word position", fontsize=24)
for row in word_order_df.iterrows():
    curr_n_src = int(row[1][0])
    curr_amp = row[1][1]
#     curr_prop = row[1][2].arr/10 # n repetitions * n blocks/rate
    row_i = subplot_row_dict[curr_n_src]
    col_j = subplot_col_dict[curr_amp]
    axes[row_i, col_j].bar(x_pos, curr_prop, align="center", width=0.8,
                           alpha=0.7, color=color_dict[curr_n_src])
    
    axes[row_i, col_j].set_xticks(x_pos)
    if row_i == 2:
        axes[row_i, col_j].set_xlabel("Target amplitude = {:.1f} deg".format(curr_amp), fontsize=14)
    else:
        axes[row_i, col_j].set_xticklabels(())
    if col_j == 0:
        axes[row_i, col_j].set_ylabel("{:d} talkers".format(curr_n_src), fontsize=14)
    else:
        axes[row_i, col_j].set_yticklabels(())
    axes[row_i, col_j].set_ylim((0, 1))

    axes[row_i, col_j].grid(linestyle=":")

fig.text(0.5, 0.05,
         "Word position", ha="center", fontsize=20)
fig.text(0.05, 0.5,
         "Proportion correct", va="center", fontsize=20, rotation=90)

plt.show()
# plt.savefig("c.pdf")

### TO DO
* Full model: n_talkers x rate x init_theta x word_pos

1 cycle is 80 deg (amplitude of 20 deg):
* 0.1 Hz = 8 deg/s

* 0.5 Hz = 40 deg/s

* 2.5 Hz = 200 deg/s

# Pilot v6

In [ ]:
def extract_condition_data(merged_data, cond):
    from functools import partial, reduce
    inner_merge = partial(pd.merge, how="inner")
    flatten = lambda t: [item for sublist in t for item in sublist] # flattens list of lists in double loop

    # Choose the subset of the stimulus database that satisfies the current conditions
    conditions = []
    conditions.append(merged_data[(merged_data["stim_type"] == cond.stim_type)])
    conditions.append(merged_data[(merged_data["is_target"] == True) &
                                  (merged_data["alt_rate"] == cond.target_alt_rate)])
    conditions.append(merged_data[(merged_data["is_target"] == False) &
                                  (merged_data["alt_rate"] == cond.masker_alt_rate)])
    if cond.target_init_angle: # if initial angle is specified in conditions, it will be not None
        conditions.append(merged_data[(merged_data["is_target"] == True) &
                                      (np.abs(merged_data["init_angle"]) == cond.target_init_angle)])
    if cond.masker_init_angle:
        conditions.append(merged_data[(merged_data["is_target"] == False) &
                                      (np.abs(merged_data["init_angle"]) == cond.masker_init_angle)])
    subsets = [merged_data.loc[merged_data["stim_num"].isin(cond["stim_num"])]
               for cond in conditions]
    conditioned_data = reduce(inner_merge, subsets)
    conditioned_data = conditioned_data[conditioned_data["is_target"]]
    return conditioned_data

In [ ]:
stim_data = pd.read_csv(PROJ_DIR/"assets"/"stimuli"/"stimulus_database.csv")
runs_to_plot = sorted(list(DATA_DIR.glob("*.csv")))

run_storage = pd.DataFrame()
for run_csv in runs_to_plot:
    run_storage = run_storage.append(pd.read_csv(run_csv))
merged_data = pd.merge(run_storage, stim_data, how="inner").reset_index(drop=True)

subject_list = np.sort(merged_data["subject_ID"].unique())
subject_list = [subj for subj in subject_list if "TRAIN" not in subj]
symbols = ["o", "s", "^", "v"]
subj_symb_map = dict(zip(subject_list, symbols[:len(subject_list)]))

control_em = [extract_condition_data(merged_data, cond) for cond in [PILOT_V6_CONTROLS[0]]]
expt_ev_em = [extract_condition_data(merged_data, cond) for cond in PILOT_V6_EV_EM]
expt_dv_em = [extract_condition_data(merged_data, cond) for cond in PILOT_V6_DV_EM]
control_im = [extract_condition_data(merged_data, cond) for cond in [PILOT_V6_CONTROLS[1]]]
expt_ev_im = [extract_condition_data(merged_data, cond) for cond in PILOT_V6_EV_IM]
expt_dv_im = [extract_condition_data(merged_data, cond) for cond in PILOT_V6_DV_IM]

In [ ]:
# Plot parameters
chance = 1/8

cyan   = (  0/256, 173/256, 239/256, 0.75)
violet = (128/256,   0/256, 128/256, 0.75)

x_min, x_max, x_spacing = -0.25, 4.25, 1
y_min, y_max, y_spacing = 0, 1, 0.1
x_ticks = np.arange(0, 5, x_spacing)
y_ticks = np.arange(y_min + y_spacing, y_max, y_spacing)

dv0 = r"""Static
"""
dv1 = r"""0.1
"""
dv2 = r"""0.5
"""
dv3 = r"""1.0
"""
dv4 = r"""2.0
"""
ev0 = r"""Static
"""
ev1 = r"""0.1
"""
ev2 = r"""0.5
"""
ev3 = r"""1.0
"""
ev4 = r"""2.0
"""
dv_x_ticklabels = [dv0, dv1, dv2, dv3, dv4]
ev_x_ticklabels = [ev0, ev1, ev2, ev3, ev4]

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 12), constrained_layout=True)

ev_em = np.empty((len(subject_list), 5))
dv_em = np.empty((len(subject_list), 5))
ev_im = np.empty((len(subject_list), 5))
dv_im = np.empty((len(subject_list), 5))
for subj in subject_list:
    curr_subj_control_em = [df[df["subject_ID"] == subj] for df in control_em]
    curr_subj_expt_ev_em = [df[df["subject_ID"] == subj] for df in expt_ev_em]
    curr_subj_expt_dv_em = [df[df["subject_ID"] == subj] for df in expt_dv_em]
    curr_subj_control_im = [df[df["subject_ID"] == subj] for df in control_im]
    curr_subj_expt_ev_im = [df[df["subject_ID"] == subj] for df in expt_ev_im]
    curr_subj_expt_dv_im = [df[df["subject_ID"] == subj] for df in expt_dv_im]
    curr_control_em_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_control_em])
    curr_expt_ev_em_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_ev_em])
    curr_expt_dv_em_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_dv_em])
    curr_control_im_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_control_im])
    curr_expt_ev_im_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_ev_im])
    curr_expt_dv_im_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_dv_im])

    axes[0, 0].plot(x_ticks[0 ], curr_control_em_corr,    color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[0, 0].plot(x_ticks[1:], curr_expt_ev_em_corr,    color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 0].plot(x_ticks[0 ], curr_control_em_corr,    color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 0].plot(x_ticks[1:], curr_expt_dv_em_corr,    color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[0, 1].plot(x_ticks[0 ], curr_control_im_corr,    color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[0, 1].plot(x_ticks[1:], curr_expt_ev_im_corr,    color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 1].plot(x_ticks[0 ], curr_control_im_corr,    color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 1].plot(x_ticks[1:], curr_expt_dv_im_corr,    color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)

fig.suptitle(r"\textbf{Word identification performance}", fontsize=24)
axes[0, 0].set_title(r"(Noise Masker)",  fontsize=18)
axes[0, 1].set_title(r"(Speech Masker)", fontsize=18)
for i, ax_rows in enumerate(axes):
    for j, ax in enumerate(ax_rows):
        ax.set_xticks(x_ticks)
        if i == 0:
#             ax.set_xticklabels(dv_x_ticklabels, fontsize=12)
            ax.set_xticklabels([], fontsize=12)
        else:
            ax.set_xticklabels(ev_x_ticklabels, fontsize=12)
        ax.set_yticks(y_ticks)
        if j == 0:
            ax.set_yticklabels(["{:.1f}".format(i) for i in y_ticks], fontsize=16)
        else:
            ax.set_yticklabels([])
        ax.tick_params(axis="both", length=0)
        ax.grid(linestyle="-", alpha=0.8)
        ax.hlines(chance, x_min, x_max, colors="k", linestyles=":")
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
fig.text( 0.5 , -0.01, r"\textbf{Target rate [Hz]}", ha="center", rotation="horizontal", fontsize=18)
fig.text(-0.03,  0.5 , r"\textbf{Proportion correct}",  va="center", rotation="vertical",   fontsize=18)

plt.show()
# fig.savefig("data_pilot_v6.pdf", bbox_inches="tight")

# Pilot v5

Load data and calculate desired dataframes:

In [ ]:
stim_data = pd.read_csv(PROJ_DIR/"archive"/"pilot_v5"/"stimuli"/"stimulus_database.csv")
runs_to_plot = sorted(list((PROJ_DIR/"archive"/"pilot_v5"/"data").glob("*.csv")))

run_storage = pd.DataFrame()
for run_csv in runs_to_plot:
    run_storage = run_storage.append(pd.read_csv(run_csv))
merged_data = pd.merge(run_storage, stim_data, how="inner").reset_index(drop=True)

subject_list = np.sort(merged_data["subject_ID"].unique())
subject_list = [subj for subj in subject_list if "TRAIN" not in subj]
symbols = ["o", "s", "^", "v"]
subj_symb_map = dict(zip(subject_list, symbols[:len(subject_list)]))




CONTROL_EM = [Condition("SEM", 0. , 0. , 45. , 45. ),
              Condition("SEM", 0. , 0. , 63.5, 63.5)]
EXPT_EV_EM = [Condition("SEM", 0.5, 0.5, None, None),
              Condition("SEM", 1. , 1. , None, None),
              Condition("SEM", 2. , 2. , None, None),
              Condition("SEM", 5. , 5. , None, None)]
EXPT_DV_EM = [Condition("SEM", 0.5, 5. , None, None),
              Condition("SEM", 1. , 2. , None, None),
              Condition("SEM", 2. , 1. , None, None),
              Condition("SEM", 5. , 0.5, None, None)]
CONTROL_IM = [Condition("SIM", 0. , 0. , 45. , 45. ),
              Condition("SIM", 0. , 0. , 63.5, 63.5)]
EXPT_EV_IM = [Condition("SIM", 0.5, 0.5, None, None),
              Condition("SIM", 1. , 1. , None, None),
              Condition("SIM", 2. , 2. , None, None),
              Condition("SIM", 5. , 5. , None, None)]
EXPT_DV_IM = [Condition("SIM", 0.5, 5. , None, None),
              Condition("SIM", 1. , 2. , None, None),
              Condition("SIM", 2. , 1. , None, None),
              Condition("SIM", 5. , 0.5, None, None)]
TRAIN_COND = [Condition("SEM", 0. , 0. , 63.5, 63.5),
              Condition("SEM", 1. , 1. , None, None),
              Condition("SEM", 1,   2. , None, None),
              Condition("SEM", 0. , 0. , 45. , 45. ),
              Condition("SIM", 0.5, 0.5, None, None),
              Condition("SIM", 0.5, 5. , None, None)]
ALL_CONDITIONS = CONTROL_EM + EXPT_EV_EM + EXPT_DV_EM \
               + CONTROL_IM + EXPT_EV_IM + EXPT_DV_IM

control_em = [extract_condition_data(merged_data, cond) for cond in CONTROL_EM]
expt_ev_em = [extract_condition_data(merged_data, cond) for cond in EXPT_EV_EM]
expt_dv_em = [extract_condition_data(merged_data, cond) for cond in EXPT_DV_EM]
control_im = [extract_condition_data(merged_data, cond) for cond in CONTROL_IM]
expt_ev_im = [extract_condition_data(merged_data, cond) for cond in EXPT_EV_IM]
expt_dv_im = [extract_condition_data(merged_data, cond) for cond in EXPT_DV_IM]

In [ ]:
# Plot parameters
chance = 1/8

cyan   = (  0/256, 173/256, 239/256, 0.75)
violet = (128/256,   0/256, 128/256, 0.75)

x_min, x_max, x_spacing = -0.25, 4.25, 1
y_min, y_max, y_spacing = 0, 1, 0.1
x_ticks = np.arange(0, 5, x_spacing)
y_ticks = np.arange(y_min + y_spacing, y_max, y_spacing)

dv0 = r"""T/ST
M/ST
($\Delta\theta=127^{\circ}$)
"""
dv1 = r"""T/VS
M/VS
"""
dv2 = r"""T/SL
M/SL
"""
dv3 = r"""T/FS
M/FS
"""
dv4 = r"""T/VF
M/VF
"""
ev0 = r"""T/ST
M/ST
($\Delta\theta=90^{\circ}$)
"""
ev1 = r"""T/VS
M/VF
"""
ev2 = r"""T/SL
M/FS
"""
ev3 = r"""T/FS
M/SL
"""
ev4 = r"""T/VF
M/VS
"""
dv_x_ticklabels = [dv0, dv1, dv2, dv3, dv4]
ev_x_ticklabels = [ev0, ev1, ev2, ev3, ev4]

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(8, 12), constrained_layout=True)

ev_em = np.empty((len(subject_list), 5))
dv_em = np.empty((len(subject_list), 5))
ev_im = np.empty((len(subject_list), 5))
dv_im = np.empty((len(subject_list), 5))
for subj in subject_list:
    curr_subj_control_em = [df[df["subject_ID"] == subj] for df in control_em]
    curr_subj_expt_ev_em = [df[df["subject_ID"] == subj] for df in expt_ev_em]
    curr_subj_expt_dv_em = [df[df["subject_ID"] == subj] for df in expt_dv_em]
    curr_subj_control_im = [df[df["subject_ID"] == subj] for df in control_im]
    curr_subj_expt_ev_im = [df[df["subject_ID"] == subj] for df in expt_ev_im]
    curr_subj_expt_dv_im = [df[df["subject_ID"] == subj] for df in expt_dv_im]
    curr_control_em_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_control_em])
    curr_expt_ev_em_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_ev_em])
    curr_expt_dv_em_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_dv_em])
    curr_control_im_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_control_im])
    curr_expt_ev_im_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_ev_im])
    curr_expt_dv_im_corr = np.array([(data["correct"]/4).mean() for data in curr_subj_expt_dv_im])

    axes[0, 0].plot(x_ticks[0 ], curr_control_em_corr[1], color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[0, 0].plot(x_ticks[1:], curr_expt_ev_em_corr,    color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 0].plot(x_ticks[0 ], curr_control_em_corr[0], color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 0].plot(x_ticks[1:], curr_expt_dv_em_corr,    color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[0, 1].plot(x_ticks[0 ], curr_control_im_corr[1], color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[0, 1].plot(x_ticks[1:], curr_expt_ev_im_corr,    color=cyan,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 1].plot(x_ticks[0 ], curr_control_im_corr[0], color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)
    axes[1, 1].plot(x_ticks[1:], curr_expt_dv_im_corr,    color=violet,
                    marker=subj_symb_map[subj], markeredgecolor="k", markeredgewidth=1.25, markersize=8)

fig.suptitle(r"\textbf{Word identification performance}", fontsize=24)
axes[0, 0].set_title(r"(Noise Masker)",  fontsize=18)
axes[0, 1].set_title(r"(Speech Masker)", fontsize=18)
for i, ax_rows in enumerate(axes):
    for j, ax in enumerate(ax_rows):
        ax.set_xticks(x_ticks)
        if i == 0:
            ax.set_xticklabels(dv_x_ticklabels, fontsize=12)
        else:
            ax.set_xticklabels(ev_x_ticklabels, fontsize=12)
        ax.set_yticks(y_ticks)
        if j == 0:
            ax.set_yticklabels(["{:.1f}".format(i) for i in y_ticks], fontsize=16)
        else:
            ax.set_yticklabels([])
        ax.tick_params(axis="both", length=0)
        ax.grid(linestyle="-", alpha=0.8)
        ax.hlines(chance, x_min, x_max, colors="k", linestyles=":")
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
fig.text( 0.5 , -0.01, r"\textbf{Movement conditions}", ha="center", rotation="horizontal", fontsize=18)
fig.text(-0.03,  0.5 , r"\textbf{Proportion correct}",  va="center", rotation="vertical",   fontsize=18)

plt.show()
# fig.savefig("data_pilot_v5.pdf", bbox_inches="tight")

# Pilot v4

In [ ]:
stim_data = pd.read_csv(PROJ_DIR/"archive"/"pilot_v4"/"stimuli"/"stimulus_database.csv")
runs_to_plot = range(7)

run_storage = pd.DataFrame()
for run_num in runs_to_plot:
    run_file_name = "RUN_" + str(run_num).zfill(3) + ".csv"
    run_storage = run_storage.append(pd.read_csv(DATA_DIR/run_file_name))
merged_data = pd.merge(run_storage, stim_data, how="inner").reset_index(drop=True)

In [ ]:
def extract_condition_data(merged_data, cond):
    from functools import partial, reduce
    inner_merge = partial(pd.merge, how="inner")
    flatten = lambda t: [item for sublist in t for item in sublist] # flattens list of lists in double loop

    # Choose the subset of the stimulus database that satisfies the current conditions
    conditions = []
    conditions.append(merged_data[(merged_data["stim_type"] == cond.stim_type)])
    conditions.append(merged_data[(merged_data["is_target"] == True) &
                                  (merged_data["alt_rate"] == cond.target_alt_rate)])
    conditions.append(merged_data[(merged_data["is_target"] == False) &
                                  (merged_data["alt_rate"] == cond.masker_alt_rate)])
    if cond.target_init_angle: # if initial angle is specified in conditions, it will be not None
        conditions.append(merged_data[(merged_data["is_target"] == True) &
                                      (np.abs(merged_data["init_angle"]) == cond.target_init_angle)])
    if cond.masker_init_angle:
        conditions.append(merged_data[(merged_data["is_target"] == False) &
                                      (np.abs(merged_data["init_angle"]) == cond.masker_init_angle)])
    subsets = [merged_data.loc[merged_data["stim_num"].isin(cond["stim_num"])]
               for cond in conditions]
    conditioned_data = reduce(inner_merge, subsets)
    conditioned_data = conditioned_data[conditioned_data["is_target"]]
    return conditioned_data

In [ ]:
expt111_data_by_cond = [extract_condition_data(merged_data, cond) for cond in EXPT_1_1_1]
expt112_data_by_cond = [extract_condition_data(merged_data, cond) for cond in EXPT_1_1_2]
expt121_data_by_cond = [extract_condition_data(merged_data, cond) for cond in EXPT_1_2_1]
expt122_data_by_cond = [extract_condition_data(merged_data, cond) for cond in EXPT_1_2_2]
expt131_data_by_cond = [extract_condition_data(merged_data, cond) for cond in EXPT_1_3_1]
expt132_data_by_cond = [extract_condition_data(merged_data, cond) for cond in EXPT_1_3_2]
expt212_data_by_cond = [extract_condition_data(merged_data, cond) for cond in EXPT_2_1_2]

expt111_corr = np.array([(data["correct"]/4).mean() for data in expt111_data_by_cond])
expt112_corr = np.array([(data["correct"]/4).mean() for data in expt112_data_by_cond])
expt121_corr = np.array([(data["correct"]/4).mean() for data in expt121_data_by_cond])
expt122_corr = np.array([(data["correct"]/4).mean() for data in expt122_data_by_cond])
expt131_corr = np.array([(data["correct"]/4).mean() for data in expt131_data_by_cond])
expt132_corr = np.array([(data["correct"]/4).mean() for data in expt132_data_by_cond])
expt212_corr = np.array([(data["correct"]/4).mean() for data in expt212_data_by_cond])

In [ ]:
legend0 = []
legend0.append(Patch(facecolor="r", edgecolor="r", label="Antiphasic"))
legend0.append(Patch(facecolor="b", edgecolor="b", label="Static target"))
legend0.append(Patch(facecolor="g", edgecolor="g", label="Static masker"))
legend0.append(Line2D([], [], linestyle="", marker="d", markeredgewidth=1.5,
                      markeredgecolor="k", markerfacecolor="w", markersize=12,
                      label="Noise masker"))
legend0.append(Line2D([], [], linestyle="", marker="s", markeredgewidth=1.5,
                      markeredgecolor="k", markerfacecolor="w", markersize=12,
                      label="Speech masker"))
legend0.append(Line2D([], [], linestyle="", marker="^", markeredgewidth=1.5,
                      markeredgecolor="k", markerfacecolor="k", markersize=12,
                      label=r"$\pm 90^{\circ}$"))
legend0.append(Line2D([], [], linestyle="", marker="v", markeredgewidth=1.5,
                      markeredgecolor="k", markerfacecolor="k", markersize=12,
                      label=r"$0^{\circ}$"))
legend0.append(Line2D([], [], linestyle="", marker="o", markeredgewidth=1.5,
                      markeredgecolor="k", markerfacecolor="k", markersize=12,
                      label="0.5 vs 5 Hz"))

# legend_elements.append(Patch(facecolor="b", edgecolor="b", label=r"$\pm 90^{\circ}$"),)
# legend_elements.append(Patch(facecolor="r", edgecolor="r", label=r"$0^{\circ}$"))
# legend_elements.append(Line2D([], [], color="k", linestyle=":", label="chance"))

In [ ]:
delta_w = 0.1
pilot_v3_pm = [0.9875, 0.9625]
pilot_v3_coloc = [0.85, 0.775]
x_locs = np.arange(1, 5)
y_locs = np.arange(0.5, 1.01, 0.1)
x_ticklabels = [0.5, 1, 2, 5]
y_ticklabels = ["{:.1f}".format(y_val) for y_val in y_locs]
y_ticks_ax1 = np.linspace(-0.3, 0.3, 7, endpoint=True)
y_ticklabels_ax1 = ["{:+.1f}".format(y_val) for y_val in y_ticks_ax1]

fig, axes = plt.subplots(1, 2, figsize=(16, 8))
axes[0].plot([3*delta_w, 3*delta_w], pilot_v3_pm, "k^", markersize=12, alpha=0.75)
axes[0].plot([4*delta_w, 4*delta_w], pilot_v3_coloc, "kv", markersize=12, alpha=0.75)
axes[0].plot([5*delta_w, 5*delta_w], expt212_corr , "ko", markersize=12, alpha=0.75)
axes[0].plot(x_locs - 1*delta_w, expt111_corr, "rd--", markersize=12, alpha=0.75)
axes[0].plot(x_locs + 0*delta_w, expt121_corr, "bd--", markersize=12, alpha=0.75)
axes[0].plot(x_locs + 1*delta_w, expt131_corr, "gd--", markersize=12, alpha=0.75)
axes[0].plot(x_locs - 1*delta_w, expt112_corr, "rs:",  markersize=12, alpha=0.75)
axes[0].plot(x_locs + 0*delta_w, expt122_corr, "bs:",  markersize=12, alpha=0.75)
axes[0].plot(x_locs + 1*delta_w, expt132_corr, "gs:",  markersize=12, alpha=0.75)

axes[0].set_title("Words correct - one target, one masker", fontsize=26)
axes[0].set_xlabel("Oscillation rate [Hz]", fontsize=24)
axes[0].set_ylabel("Proportion correct", fontsize=24)
axes[0].set_xlim((0, x_locs[-1] + 5*delta_w))
axes[0].set_ylim((0.5, 1.01))
axes[0].set_xticks(x_locs)
axes[0].set_yticks(y_locs)
axes[0].set_xticklabels(x_ticklabels, fontsize=20)
axes[0].set_yticklabels(y_ticklabels, fontsize=20)
axes[0].grid(color="k", linestyle="-", alpha=0.25)
axes[0].legend(handles=legend0, loc=0, fontsize=18)

axes[1].bar(x_locs + 0*delta_w, expt112_corr - expt111_corr,
            width=delta_w, color="r", align="edge", alpha=0.75)
axes[1].bar(x_locs + 1*delta_w, expt122_corr - expt121_corr,
            width=delta_w, color="b", align="edge", alpha=0.75)
axes[1].bar(x_locs + 2*delta_w, expt132_corr - expt131_corr,
            width=delta_w, color="g", align="edge", alpha=0.75)
axes[1].hlines(0, 1*delta_w, x_locs[-1] + 4*delta_w, color="k", alpha=0.8)

axes[1].set_title("How much easier is SOS vs noise? (i.e. EM vs IM)", fontsize=26)
axes[1].set_xlabel("Oscillation rate [Hz]", fontsize=24)
axes[1].set_ylabel(r"$\Delta$ prop. correct", fontsize=24)
axes[1].set_xlim((x_locs[0] - 1*delta_w, x_locs[-1] + 4*delta_w))
axes[1].set_ylim((-0.31, 0.31))
axes[1].set_xticks(x_locs + 1.5*delta_w)
axes[1].set_yticks(y_ticks_ax1)
axes[1].set_xticklabels(x_ticklabels, fontsize=20)
axes[1].set_yticklabels(y_ticklabels_ax1, fontsize=20)
axes[1].grid(color="k", linestyle="-", alpha=0.25)

fig.tight_layout()
fig.savefig("pilot_v4.pdf")

# Pilot v3 - two or three talkers; also examining randomization effect
* Red - two talkers, conditions (alternation rates) are blocked
* Blue - two talkers, conditions are randomized within block
* Yellow - three talkers, conditions are blocked
* Green - three talkers, conditions are randomized within block

In [ ]:
stim_data = pd.read_csv(PROJ_DIR/"archive"/"pilot_v3"/"stimuli"/"stimulus_database.csv")
runs_to_plot = range(4)

run_storage = pd.DataFrame()
for run_num in runs_to_plot:
    run_file_name = "RUN_" + str(run_num).zfill(3) + ".csv"
    run_storage = run_storage.append(pd.read_csv(PROJ_DIR/"archive"/"pilot_v3"/"data"/run_file_name))
merged_data = pd.merge(run_storage, stim_data, how="inner").reset_index(drop=True)

In [ ]:
chance_level = 1/8
colors = ["r", "b", "y", "g", "m", "c", "y", "k"]
marker_symbols = ["o", "v", "^", "d", "*", "x", "s", "p"]
conditions = ["colocated", "plus_minus_90", 0.5, 2, 4, 8]
x_tick_locs = [-0.5, 9, 0.5, 2, 4, 8]
x_tick_labels = [(r"$\pm90^{\circ}$"), "co-loc", "0.5", "2", "4", "8"]

# legend_elements = []
# for i in range(n_runs_to_load):
#     legend_elements = [Line2D([], [], linestyle="", marker=marker_symbols[i], markeredgewidth=3, markeredgecolor="k", markerfacecolor="w", markersize=18, label=subjects[i])]
# legend_elements.append(Patch(facecolor="b", edgecolor="b", label=r"$\pm 90^{\circ}$"),)
# legend_elements.append(Patch(facecolor="r", edgecolor="r", label=r"$0^{\circ}$"))
# legend_elements.append(Line2D([], [], color="k", linestyle=":", label="chance"))

fig, ax = plt.subplots(1, 1, figsize=(12, 9))

for idx, run_num in enumerate(runs_to_plot):
    curr_run = merged_data[(merged_data["is_target"] == True) &
                           (merged_data["run_num"] == run_num)]
    rate_correct = []
    for cond in conditions:
        if cond == "colocated":
            curr_cond = curr_run[(curr_run["alt_rate"] == 0) &
                                 (curr_run["init_angle"] == 0)]
            n_correct = sum(curr_cond["correct"])
            n_total = 4*len(curr_cond)
            ax.plot(x_tick_locs[1], n_correct/n_total, colors[idx] + marker_symbols[idx])
        elif cond == "plus_minus_90":
            curr_cond = curr_run[(curr_run["alt_rate"] == 0) &
                                 (curr_run["init_angle"] != 0)]
            n_correct = sum(curr_cond["correct"])
            n_total = 4*len(curr_cond)
            ax.plot(x_tick_locs[0], n_correct/n_total, colors[idx] + marker_symbols[idx])
        else:
            curr_cond = curr_run[curr_run["alt_rate"] == cond]
            n_correct = sum(curr_cond["correct"])
            n_total = 4*len(curr_cond)
            rate_correct.append((cond, n_correct/n_total))
    rate_correct = np.array(rate_correct)
    ax.plot(rate_correct[:, 0], rate_correct[:, 1], colors[idx] + marker_symbols[idx] + "-")
ax.hlines(chance_level, x_tick_locs[0] - 0.5, x_tick_locs[1] + 0.5, color="k", linestyle=":", alpha=0.75)

ax.set_title("Word by word identification performance", fontsize=40)
ax.set_xlabel("Oscillation rate [Hz]", fontsize=28)
ax.set_ylabel("Proportion correct", fontsize=28)
ax.set_xlim((x_tick_locs[0] - 0.5, x_tick_locs[1] + 0.5))
ax.set_ylim((0, 1))
ax.set_xticks(x_tick_locs)
ax.set_yticks(np.linspace(0, 1, 11, endpoint=True))
ax.set_xticklabels(x_tick_labels, fontsize=20)
ax.set_yticklabels(np.linspace(0, 1, 11, endpoint=True), fontsize=20)
ax.yaxis.set_major_formatter(FormatStrFormatter("%.1f"))

ax.grid(linestyle="-")
# ax.legend(handles=legend_elements)

# plt.savefig("ayc_pilot_v3.pdf")
plt.show()

### Plot proportion correct by talker

In [ ]:
grouped_by_talker_2 = merged_data[(merged_data["is_target"] == True) &
                                  (merged_data["n_srcs"] == 2)].groupby("src").sum("correct")
correct_by_talker_2 = grouped_by_talker_2["correct"]
possible_correct_2 = 4*grouped_by_talker_2["is_target"].values
prop_correct_by_talker_2 = correct_by_talker_2/possible_correct_2
prop_correct_by_talker_2 = prop_correct_by_talker_2.sort_values(ascending=False)

grouped_by_talker_3 = merged_data[(merged_data["is_target"] == True) &
                                  (merged_data["n_srcs"] == 3)].groupby("src").sum("correct")
correct_by_talker_3 = grouped_by_talker_3["correct"]
possible_correct_3 = 4*grouped_by_talker_3["is_target"].values
prop_correct_by_talker_3 = correct_by_talker_3/possible_correct_3
prop_correct_by_talker_3 = prop_correct_by_talker_3.sort_values(ascending=False)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
prop_correct_by_talker_2.plot(kind="bar", ylim=(0, 1), ax=axes[0])
prop_correct_by_talker_3.plot(kind="bar", ylim=(0, 1), ax=axes[1])

axes[0].set_title("Two talkers")
axes[1].set_title("Three talkers")
axes[1].set_yticklabels([])

plt.tight_layout()
# plt.show()

# Pilot v2 - three talkers, one co-located masker, one antiphasic masker

In [ ]:
run_figsize = (12, 8)
n_runs_to_load = 1
n_trials_per_block = 8
n_words_per_trial = 4
chance_level = 1/8

marker_symbols = ["o", "v", "^", "d", "*"]
subjects = ["ayc"]
legend_elements = []
for i in range(n_runs_to_load):
    legend_elements = [Line2D([], [], linestyle="", marker=marker_symbols[i], markeredgewidth=3, markeredgecolor="k", markerfacecolor="w", markersize=18, label=subjects[i])]
legend_elements.append(Patch(facecolor="b", edgecolor="b", label=r"$\pm 90^{\circ}$"),)
legend_elements.append(Patch(facecolor="r", edgecolor="r", label=r"$0^{\circ}$"))
legend_elements.append(Line2D([], [], color="k", linestyle=":", label="chance"))

all_run_nums = range(n_runs_to_load)
fig, ax = plt.subplots(1, 1, figsize=run_figsize)
for run_num in all_run_nums:
    run_file_name = "RUN_" + str(run_num).zfill(3) + ".csv"
    stim_data = pd.read_csv(PROJ_DIR/"archive"/"pilot_v2"/"stimuli"/"stimulus_database.csv")
    run_data = pd.read_csv(PROJ_DIR/"archive"/"pilot_v2"/"data"/run_file_name)
    
    run_stim = stim_data.loc[run_data.stimulus_ID].reset_index()
    run_stim = run_stim.drop(labels=["index", "stim_type"], axis=1)
    run_data = run_data.join(run_stim)
    run_data = run_data.drop(labels=["run_num", "subject_ID", "task_type"], axis=1)

    n_blocks = run_data.block_num.max()
#     n_max_correct = n_trials_per_block*n_words_per_trial
    n_max_correct = n_blocks*n_trials_per_block*n_words_per_trial

    # Group data
    run_rate_grouped = run_data.groupby(by="target_alt_rate")
    rates = list(run_rate_grouped.indices.keys())[1:]
    correct   = run_rate_grouped.sum()["correct"].values[1:]
    # either ear
    antipodal = run_data[(run_data["target_alt_rate"] == 0) & \
                         (run_data["target_init_position"] != 0)]["correct"].sum()
    # co-located
    colocated = run_data[(run_data["target_alt_rate"] == 0) & \
                         (run_data["target_init_position"] == 0)]["correct"].sum()

    # Plot
    ax.plot(rates[ 0] - 1, antipodal/n_max_correct, "b" + marker_symbols[run_num], markersize=18)
    ax.plot(        rates,   correct/n_max_correct, "k" + marker_symbols[run_num] + "-", markersize=18)
    ax.plot(rates[-1] + 1, colocated/n_max_correct, "r" + marker_symbols[run_num], markersize=18)
ax.hlines(chance_level, rates[0] - 2*1.75, rates[-1] + 2*1.75, color="k", linestyle=":")

ax.set_title("Word by word identification performance", fontsize=40)
ax.set_xlabel("Oscillation rate [Hz]\n(linear scale)", fontsize=28)
ax.set_ylabel("Percent correct", fontsize=28)
ax.set_xlim((rates[0] - 1.75, rates[-1] + 1.75))
ax.set_ylim((0, 1))
ax.set_xticks(rates)
ax.set_xticklabels(rates, rotation=90, fontsize=20)
ax.set_yticks(np.linspace(0, 1, 11, endpoint=True))
ax.set_yticklabels(np.linspace(0, 1, 11, endpoint=True), fontsize=20)
ax.xaxis.set_major_formatter(FormatStrFormatter("%.1f"))
ax.yaxis.set_major_formatter(FormatStrFormatter("%.1f"))
ax.grid(linestyle="-")
ax.legend(handles=legend_elements)
# ax.grid(linestyle="--", alpha=0.5)

# plt.savefig("b.pdf")
plt.show()